In [ ]:
# setup vars/funcs for scanCore

def dprint(text, end="\r"):
    if DEBUG:
        print(text, end=end)

DEBUG = True

In [ ]:
#   creates ip ranges

import random

# create a list of ipv4 ranges
ip_lists = []
for i in range(255):
    for j in range(255):
        ip_lists.append(f"{i}.{j}.0.0/16")
random.shuffle(ip_lists)

ip_lists = ["172.64.0.0/16","172.65.0.0/16","172.66.0.0/16"] # remove for final vers

In [ ]:
#     checks if the ip is runnging a server

import json
import os
import time

upHosts = []
results = []
start = time.time()
prev = 0

for ip_list in ip_lists:
    print(f"scanning {ip_list} | List #{ip_lists.index(ip_list)+1}/{len(ip_lists)} | average time/list {(((time.time()-start)/(ip_lists.index(ip_list)+1)*2),2)} {' '*10}",end="\r")
    try:
        import masscan
        import json

        scanner = masscan.PortScanner()
        scanner.scan(ip_list, ports='25565', arguments='--max-rate 1000')
        res = json.loads(scanner.scan_result)
        results.append(list(res["scan"].keys()))
    except Exception as e:
        print(e)

    dprint(f"found {len(results[ip_lists.index(ip_list)])} hosts | List #{ip_lists.index(ip_list)+1}/{len(ip_lists)} | total hosts: {prev+len(results[ip_lists.index(ip_list)])} | average time/list {round(((time.time()-start)/(ip_lists.index(ip_list)+1)),2)} {' '*10}",end="\r")
    prev = len(results[ip_lists.index(ip_list)])
prev = 0
for result in results:
    prev += len(result)

print(f"found {prev} hosts | average time/list {round(((time.time()-start)/(len(ip_lists))),2)} {' '*10}")

In [15]:
#     checks if the ip is runnging a minecraft server

import time

start_time = time.time()
res = []
lst = []
for result in results:
    for host in result:
        lst.append(host)

def check(host):
    try:
        import mcstatus
        print("\r",end="")
        server = mcstatus.JavaServer.lookup(host)
        status = server.status()
        description = status.description
        #remove color codes and whitespace
        description = description.replace("§0","").replace("§1","").replace("§2","").replace("§3","").replace("§4","").replace("§5","").replace("§6","").replace("§7","").replace("§8","").replace("§9","").replace("§a","").replace("§b","").replace("§c","").replace("§d","").replace("§e","").replace("§f","").replace("§k","").replace("§l","").replace("§m","").replace("§n","").replace("§o","").replace("§r","").replace(" ","-")
        dprint(f"{host} ({description}): has {status.players.online} players and replied in {round(status.latency,2)} ms\n")

        res.append(host)
    except Exception as e:
        pass

import threading

print("Starting threads")
threads = []
for host in lst:
    thread = threading.Thread(target=check, args=(host,))
    thread.start()
    threads.append(thread)

# monitor threads
while True:
    time.sleep(0.1)
    if not any(t.is_alive() for t in threads):
        break


Starting threads
172.65.119.167 (BetterMinecraft[FABRIC]1.19): has 0 players and replied in 1.9242017360002137 ms
172.65.130.212 (AMinecraftServer): has 0 players and replied in 1.3646583160002592 ms
172.65.136.250 (SynapseV3SMP): has 0 players and replied in 0.19300688100020125 ms
172.65.6.243 (PixelBUDSMP): has 0 players and replied in 0.9889452939996772 ms
172.65.103.82 (Idon'tstartshit,butIcantellyouhowitendsDon'tgetsad,getevenSoontheweekendsIdon'tdressforfriendsLately,I'vebeendressin'forrevenge): has 0 players and replied in 1.5540183289999732 ms
172.65.247.199 (Invalidhostname.Pleaserefertoourdocumentationatdocs.tcpshield.com): has 0 players and replied in 0.7225640990000102 ms
172.65.96.159 (twitch.tv/t0nysama): has 0 players and replied in 0.5159285680001631 ms
172.65.119.103 (MassiveCraft-TheFactionsRPG): has 16 players and replied in 0.5807589619998907 ms
172.65.96.189 (Welcometonorwick!): has 0 players and replied in 0.33277619099999356 ms
172.65.29.215 (ServidorSMPdoArnaud


KeyboardInterrupt: 

In [ ]:
#   Deal with the data

for i in res:
    import mcstatus

    server = mcstatus.JavaServer.lookup(i)
    status = server.status()
    print(f"\r{host}: has {status.players.online} players and replied in {status.latency} ms")